## THE CIRCUIT CONFIGURATION

In [8]:
# check if notebook is in colab
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import json
import ezkl
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import os
import sys
import sys

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)
from training.defineSNN import MODSiameseBambooNN


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


# Specify all the files we need
model_path = os.path.join('ezkl-outputs/snn.onnx')
compiled_model_path = os.path.join('ezkl-outputs/snn.ezkl')
pk_path = os.path.join('ezkl-outputs/pk.key')
vk_path = os.path.join('ezkl-outputs/vk.key')
settings_path = os.path.join('ezkl-outputs/settings.json')
srs_path = os.path.join('ezkl-outputs/kzg.srs')
data_path = os.path.join('ezkl-outputs/input.json')
cal_path = os.path.join('ezkl-outputs/cal_data.json')
##
sol_code_path = os.path.join('../contracts/Verifier.sol')
abi_path = os.path.join('ezkl-outputs/Verifier.abi')
proof_path = os.path.join('ezkl-outputs/proof.json')



In [9]:
### Load the image pair as input ready for inference
class SiameseImageLoader:
    def __init__(self, transform=None):
        self.transform = transform

    def load_and_transform_pair(self, image1_path, image2_path):
        img1 = Image.open(image1_path).convert('L')
        img2 = Image.open(image2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        img1 = img1.to(device)
        img2 = img2.to(device)

        return img1, img2

transform = transforms.Compose([
    transforms.Resize((10, 10)),
    transforms.ToTensor(),
])

# Example inputs (TrueNegative, TruePositive, FalseNegative)
# (FalseNegative were produced with a growing rate of bamboo higher than what the model was trained on)
siamese_loader = SiameseImageLoader(transform)
image1_path = "TestValid_t1.jpeg"
image2_path = "TestValid_t0.jpeg"
img1, img2 = siamese_loader.load_and_transform_pair(image1_path, image2_path) # Separate tensors for each image
img1 = img1.unsqueeze(0)
img2 = img2.unsqueeze(0)

print(img1.size())

torch.Size([1, 1, 10, 10])


In [11]:
# Load the trained nn model and make inference on test inputs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = MODSiameseBambooNN().to(device)
loaded_model.load_state_dict(torch.load('../training/trained_simplesnn_lr_0.01.pth', map_location=torch.device('cpu')))
loaded_model.eval()

with torch.no_grad():
    finalout = loaded_model(img1, img2)
    predicted_label = (torch.sigmoid(finalout) > 0.5).item()

print(f'Same class probability: {torch.sigmoid(finalout)}')
print(f'Predicted Label: {predicted_label}')

# Calculate the total number of parameters
total_params = sum(p.numel() for p in loaded_model.parameters())
print(f'Total Parameters: {total_params}')

Same class probability: tensor([[0.6949]])
Predicted Label: True
Total Parameters: 30849


In [12]:
# Export the loaded Siamese Network model to ONNX
torch.onnx.export(loaded_model,                      # loaded Siamese Network model
                  (img1, img2),                      # model input (or a tuple for multiple inputs)
                  model_path,                        # where to save the ONNX model
                  export_params=True,                # store the trained parameter weights inside the model file
                  opset_version=14,                  # the ONNX version to export the model to
                  do_constant_folding=True,          # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})



# Move tensors from GPU to CPU
img1_cpu = img1.cpu()
img2_cpu = img2.cpu()
print(img1_cpu.size())

# Serialize data into a JSON file
input_data = [
    img1_cpu.detach().numpy().reshape([-1]).tolist(),
    img2_cpu.detach().numpy().reshape([-1]).tolist(),
]

data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))

torch.Size([1, 1, 10, 10])


In [16]:
# *** Providing the calibrate settings function with a larger and broader range of sample inputs.


csv_path = '../training/dataset/forest_dataset.csv'
df = pd.read_csv(csv_path)

# Construct the full paths to the images
img1_files = [os.path.join('..', 'training', path) for path in df['imageT0'].iloc[:20]]
img2_files = [os.path.join('..', 'training', path) for path in df['imageT1'].iloc[:20]]

img1_list = []
img2_list = []
for img1_path, img2_path in zip(img1_files, img2_files):
    img1, img2 = siamese_loader.load_and_transform_pair(img1_path, img2_path)
    img1 = img1.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img2 = img2.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img1_list.extend(img1) # don't use .append bc it should look like [[xxx...],[yyy...]]
    img2_list.extend(img2)

data = dict(input_data = [img1_list, img2_list])
# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

In [17]:
#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True
print("gen_settings OK")

# *** SCALES is one of main knobs you can turn to trade off accuracy for proving efficiency
# Under the hood calibration iterates over the scales array to see how precise you can go before failure. 
# For example if 7 fails it falls back to 1.
# (Right now the default scales values for the resources target is 8-10 and 10-13  for accuracy)
res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", scales = [1, 7])
assert res == True

INFO ezkl.graph.model 2023-12-22 17:20:59,458 model.rs:724 set batch_size to 1
INFO ezkl.graph.model 2023-12-22 17:20:59,542 model.rs:440 model has 1 instances
INFO ezkl.graph.model 2023-12-22 17:20:59,547 model.rs:1330 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2023-12-22 17:21:00,092 model.rs:1403 model uses 109395 rows (coord=109395, constants=92921)
INFO ezkl.graph.model 2023-12-22 17:21:00,139 model.rs:724 set batch_size to 1
INFO ezkl.execute 2023-12-22 17:21:00,208 execute.rs:634 num of calibration batches: 20
INFO ezkl.graph.model 2023-12-22 17:21:00,233 model.rs:724 set batch_size to 1
INFO ezkl.graph.model 2023-12-22 17:21:00,271 model.rs:440 model has 1 instances
INFO ezkl.graph.model 2023-12-22 17:21:00,273 model.rs:1330 calculating num of constraints using dummy model layout...


gen_settings OK


INFO ezkl.graph.model 2023-12-22 17:21:00,542 model.rs:1403 model uses 48009 rows (coord=48009, constants=32433)
INFO ezkl.graph 2023-12-22 17:21:00,548 mod.rs:685 input scales: [1, 1]
INFO ezkl.graph.model 2023-12-22 17:21:00,731 model.rs:440 model has 1 instances
INFO ezkl.graph.model 2023-12-22 17:21:00,733 model.rs:1330 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2023-12-22 17:21:00,986 model.rs:1403 model uses 48009 rows (coord=48009, constants=32433)
INFO ezkl.graph 2023-12-22 17:21:00,992 mod.rs:938 setting lookup_range to: (-74, 82), setting logrows to: 16
INFO ezkl.graph.model 2023-12-22 17:21:01,006 model.rs:724 set batch_size to 1
INFO ezkl.graph.model 2023-12-22 17:21:01,040 model.rs:440 model has 1 instances
INFO ezkl.graph.model 2023-12-22 17:21:01,043 model.rs:1330 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2023-12-22 17:21:01,340 model.rs:1403 model uses 48009 rows (coord=48009, constants=32433)


In [18]:
# Now compile the model into a circuit
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

INFO ezkl.graph.model 2023-12-22 17:24:27,996 model.rs:724 set batch_size to 1


In [19]:
# get public srs from kzg ceremony, saved to srs path.
res = ezkl.get_srs(srs_path, settings_path)
assert res == True

INFO ezkl.execute 2023-12-22 17:24:42,697 execute.rs:485 SRS downloaded


In [20]:
# Setup the circuit and make sure the keys are generated afterwards.
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO ezkl.pfsys.srs 2023-12-22 17:24:44,364 srs.rs:23 loading srs from "ezkl-outputs/kzg.srs"
INFO ezkl.execute 2023-12-22 17:24:44,390 execute.rs:1998 downsizing params to 16 logrows
INFO ezkl.graph.vars 2023-12-22 17:24:44,393 vars.rs:408 number of blinding factors: 5
INFO ezkl.graph.model 2023-12-22 17:24:44,399 model.rs:1020 configuring model
INFO ezkl.graph 2023-12-22 17:24:44,410 mod.rs:1246 circuit size: 
 {
  "num_advice_columns": 3,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 15
}
WARNING ezkl.circuit.modules.planner 2023-12-22 17:24:44,444 planner.rs:116 spawning module 2


INFO ezkl.graph.model 2023-12-22 17:24:44,448 model.rs:1054 model layout...
INFO ezkl.graph.model 2023-12-22 17:24:46,395 model.rs:1137 model uses 45800 rows (coord=45800, constants=10080)
INFO ezkl.pfsys 2023-12-22 17:24:52,793 mod.rs:401 VK took 8.399
INFO ezkl.graph.vars 2023-12-22 17:24:52,795 vars.rs:408 number of blinding factors: 5
INFO ezkl.graph.model 2023-12-22 17:24:52,798 model.rs:1020 configuring model
INFO ezkl.graph 2023-12-22 17:24:52,800 mod.rs:1246 circuit size: 
 {
  "num_advice_columns": 3,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 15
}
WARNING ezkl.circuit.modules.planner 2023-12-22 17:24:52,831 planner.rs:116 spawning module 2
INFO ezkl.graph.model 2023-12-22 17:24:52,834 model.rs:1054 model layout...
INFO ezkl.graph.model 2023-12-22 17:24:54,444 model.rs:1137 model uses 45800 rows (coord=45800, constants=10080)
INFO ezkl.pfsys 2023-12-22 17:24:57,403 mod.rs:407 PK took 4.608
INFO ezkl.pfsys 2023-12-22 17:24:57,405 mod.rs:67

In [21]:
# GENERATE the verifier associated with the circuit
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])
    !solc-select install 0.8.20
    !solc-select use 0.8.20
    !solc --version
# rely on local installation if the notebook is not in colab
except:
    pass


In [22]:
res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path
    )
assert res == True
assert os.path.isfile(sol_code_path)

INFO ezkl.execute 2023-12-22 17:25:05,437 execute.rs:76 checking solc installation..
INFO ezkl.pfsys.srs 2023-12-22 17:25:05,828 srs.rs:23 loading srs from "ezkl-outputs/kzg.srs"
INFO ezkl.execute 2023-12-22 17:25:05,843 execute.rs:1998 downsizing params to 16 logrows
INFO ezkl.pfsys 2023-12-22 17:25:05,844 mod.rs:614 loading verification key from "ezkl-outputs/vk.key"
INFO ezkl.graph.vars 2023-12-22 17:25:05,845 vars.rs:408 number of blinding factors: 5
INFO ezkl.graph.model 2023-12-22 17:25:05,847 model.rs:1020 configuring model
INFO ezkl.graph 2023-12-22 17:25:05,848 mod.rs:1246 circuit size: 
 {
  "num_advice_columns": 3,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 15
}


## PROVE and VERIFY
### here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [23]:
witness_path = os.path.join('ezkl-outputs/witness.json')
# generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

INFO ezkl.graph 2023-12-22 17:25:14,326 mod.rs:706 input scales: [1, 1]


In [24]:
# !!!Generate the proof!!!

proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

print(proof)
assert os.path.isfile(proof_path)

INFO ezkl.pfsys.srs 2023-12-22 17:25:17,190 srs.rs:23 loading srs from "ezkl-outputs/kzg.srs"
INFO ezkl.execute 2023-12-22 17:25:17,235 execute.rs:1998 downsizing params to 16 logrows
INFO ezkl.pfsys 2023-12-22 17:25:17,237 mod.rs:636 loading proving key from "ezkl-outputs/pk.key"
INFO ezkl.graph.vars 2023-12-22 17:25:17,238 vars.rs:408 number of blinding factors: 5
INFO ezkl.graph.model 2023-12-22 17:25:17,240 model.rs:1020 configuring model
INFO ezkl.graph 2023-12-22 17:25:17,241 mod.rs:1246 circuit size: 
 {
  "num_advice_columns": 3,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 15
}
INFO ezkl.pfsys 2023-12-22 17:25:17,848 mod.rs:470 proof started...
INFO ezkl.graph.vars 2023-12-22 17:25:17,851 vars.rs:408 number of blinding factors: 5
INFO ezkl.graph.model 2023-12-22 17:25:17,853 model.rs:1020 configuring model
INFO ezkl.graph 2023-12-22 17:25:17,855 mod.rs:1246 circuit size: 
 {
  "num_advice_columns": 3,
  "num_challenges": 0,
  "num_fixed": 4

{'instances': [[[11874046804959952378, 12031363491236821520, 978108733370872704, 1515817019597253525]]], 'proof': '03afc4dab81a172c9249d9cd08823bf1449155d2bc9ea8488d8bc8f18d6fbf8919b629f63bd369f984955f80421baf459b783ca8e6942f4539d7d42b9120cd7c1d5d24c262bd7cc8312f0afff79d27fecb20496e2076282f3a3c1bb42de057040f97d8ab61c95505c7dbae40f9fa520cef33979863aa77413862cc32b04b6e9f200905478ad2527f014877ddf4143b4da66c80cd4ccc970433db6dd5309aa6f9032e95318a082e24ef00778238f6111fd2f65edd4da14fccb54ec7c0ce0da1e9225f2651c46329a1fc6e539d7b96ec40d4fb38d7ee6d89a3390dfca138c9d0861c4eea3c7e481565a69766721b7d54c7687a25c34f8b4953215fd60ab1d14c9f026d819e1de29433862d17eb9797049372d4519ba626b7743b7be3530fb5e402183ea17e41c4d5e00825c2bde4f01503bbcecab20bd9297fcfce4648c95d20c221002947148658ca4a137d6f241152f69edffd78262219a20ba78b4d9e43ee4c036d3b2c832e5a5459c33cef21423dca6a4f79b6c5bb2e8fce4b9098926d1c850ab2caa8707650add3cfcef00e48af5aaef65cad9995c57698362e92a38624e2273b49cb0b62ee270360a9f716c6e37f3c8deaad5c7c232b48b3e

In [25]:
# Sanity check off-chain verification
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified locally")

INFO ezkl.pfsys.srs 2023-12-22 17:25:38,270 srs.rs:23 loading srs from "ezkl-outputs/kzg.srs"
INFO ezkl.execute 2023-12-22 17:25:38,291 execute.rs:1998 downsizing params to 16 logrows
INFO ezkl.pfsys 2023-12-22 17:25:38,296 mod.rs:614 loading verification key from "ezkl-outputs/vk.key"
INFO ezkl.graph.vars 2023-12-22 17:25:38,330 vars.rs:408 number of blinding factors: 5
INFO ezkl.graph.model 2023-12-22 17:25:38,332 model.rs:1020 configuring model
INFO ezkl.graph 2023-12-22 17:25:38,336 mod.rs:1246 circuit size: 
 {
  "num_advice_columns": 3,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 15
}
INFO ezkl.execute 2023-12-22 17:25:38,485 execute.rs:1721 verify took 0.117
INFO ezkl.execute 2023-12-22 17:25:38,488 execute.rs:1726 verified: true


verified locally


In [26]:
# Get input data for the verifier contract (evmInputs.json)
onchain_input_array = []
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"
print("form output",formatted_output )
    
input_list = [int(entry, 0) for entry in formatted_output.strip('[]').split(', ')]
quoted_list = [format(entry, "#066x") for entry in input_list]
json_data = {
    "instances": quoted_list,
    "proof": "0x" + proof["proof"]
}
with open('evmInputs.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)


form output [0x0000000000000000000000000000000000000000000000000000000000000062]
Saved evmInputs.json:  {'instances': ['0x0000000000000000000000000000000000000000000000000000000000000062'], 'proof': '0x03afc4dab81a172c9249d9cd08823bf1449155d2bc9ea8488d8bc8f18d6fbf8919b629f63bd369f984955f80421baf459b783ca8e6942f4539d7d42b9120cd7c1d5d24c262bd7cc8312f0afff79d27fecb20496e2076282f3a3c1bb42de057040f97d8ab61c95505c7dbae40f9fa520cef33979863aa77413862cc32b04b6e9f200905478ad2527f014877ddf4143b4da66c80cd4ccc970433db6dd5309aa6f9032e95318a082e24ef00778238f6111fd2f65edd4da14fccb54ec7c0ce0da1e9225f2651c46329a1fc6e539d7b96ec40d4fb38d7ee6d89a3390dfca138c9d0861c4eea3c7e481565a69766721b7d54c7687a25c34f8b4953215fd60ab1d14c9f026d819e1de29433862d17eb9797049372d4519ba626b7743b7be3530fb5e402183ea17e41c4d5e00825c2bde4f01503bbcecab20bd9297fcfce4648c95d20c221002947148658ca4a137d6f241152f69edffd78262219a20ba78b4d9e43ee4c036d3b2c832e5a5459c33cef21423dca6a4f79b6c5bb2e8fce4b9098926d1c850ab2caa8707650add3cfcef00e48af